In [60]:
import pandas as pd
import matplotlib.pyplot as plt


In [61]:
df1 = pd.read_csv('/home/abhishek/Documents/ML_Projects/HousePricePrediction/Bengaluru_House_Data.csv')
df1.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [62]:
drop_columns = ['area_type','availability','society']
df2 = df1.drop(drop_columns,axis=1)


In [63]:
df2.head()

,location,size,total_sqft,bath,balcony,price
0,Electronic City Phase II,2 BHK,1056,2.0,1.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,3.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,3.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,1.0,95.00
4,Kothanur,2 BHK,1200,2.0,1.0,51.00


In [64]:
df2.isnull().sum()

location        1
size           16
total_sqft      0
bath           73
balcony       609
price           0
dtype: int64

In [94]:
df3 = df2.dropna()

In [95]:
def convert_size_into_numeric(x):
    return int(x.split(' ')[0])

In [96]:
# Transforming the size into numeric values
df3['size_numeric'] = df3['size'].apply(convert_size_into_numeric)

/tmp/ipykernel_7493/80423784.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['size_numeric'] = df3['size'].apply(convert_size_into_numeric)


In [97]:
# Function to check wheather the total_sqft is float or not
def check_sqft(x):
    try:
        float(x)
    except:
        return False
    return True


In [98]:
df4 =df3[~df3['total_sqft'].apply(check_sqft)]

In [99]:
df3

,location,size,total_sqft,bath,balcony,price,size_numeric
0,Electronic City Phase II,2 BHK,1056,2.0,1.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,3.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,3.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,1.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,1.0,51.00,2
...,...,...,...,...,...,...,...
13314,Green Glen Layout,3 BHK,1715,3.0,3.0,112.00,3
13315,Whitefield,5 Bedroom,3453,4.0,0.0,231.00,5
13317,Raja Rajeshwari Nagar,2 BHK,1141,2.0,1.0,60.00,2
13318,Padmanabhanagar,4 BHK,4689,4.0,1.0,488.00,4


In [110]:
# Function to find the average of the sqft
def sqft_avg(x):
    list_values = x.split('-')
    if len(list_values)==2:
        return((float(list_values[0])+float(list_values[1]))/2)
    try:
        return float(x)
        
    except:
        return None
        

In [112]:
df3['total_sqft'] = df3['total_sqft'].apply(sqft_avg)

/tmp/ipykernel_7493/865068035.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['total_sqft'] = df3['total_sqft'].apply(sqft_avg)


In [122]:
x=round(df3['total_sqft'].mean(),2)
df3['total_sqft'].fillna(x,inplace=True)

/home/abhishek/.local/lib/python3.8/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [128]:
df3.loc[672]

location        Bettahalsoor
size               4 Bedroom
total_sqft            4046.0
bath                     4.0
balcony                  0.0
price                  445.0
size_numeric               4
Name: 672, dtype: object